In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/shakespeare/shakespeare.txt


# Generating Shakespearean Text with Character Based RNNs

Problem Statement: Given a character or sequence of characters, we want to predict the next character at each time step. Model is trained to follow a language similar to the works of Shakespeare. The tinyshakespear dataset is used for training.

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import nltk
import os
import time

In [5]:
dataset = open('/kaggle/input/shakespeare/shakespeare.txt', 'r').read() 

In [10]:
text = dataset[37:]

In [11]:
vocabulary = list(sorted(set(text)))
print('No. of unique characters: {}'.format(len(vocabulary)))

No. of unique characters: 61


In [12]:
# text =text.lower()

## Preprocessing Text

In [13]:

char_to_index = dict((c,i) for (i,c) in enumerate(vocabulary))
print(char_to_index)
int_text = np.array([char_to_index[i] for i in text])

index_to_char = np.array(vocabulary)
print(index_to_char)

{'\n': 0, ' ': 1, '!': 2, "'": 3, '(': 4, ')': 5, ',': 6, '-': 7, '.': 8, ':': 9, ';': 10, '?': 11, 'A': 12, 'B': 13, 'C': 14, 'D': 15, 'E': 16, 'F': 17, 'G': 18, 'H': 19, 'I': 20, 'J': 21, 'K': 22, 'L': 23, 'M': 24, 'N': 25, 'O': 26, 'P': 27, 'R': 28, 'S': 29, 'T': 30, 'U': 31, 'V': 32, 'W': 33, 'Y': 34, 'a': 35, 'b': 36, 'c': 37, 'd': 38, 'e': 39, 'f': 40, 'g': 41, 'h': 42, 'i': 43, 'j': 44, 'k': 45, 'l': 46, 'm': 47, 'n': 48, 'o': 49, 'p': 50, 'q': 51, 'r': 52, 's': 53, 't': 54, 'u': 55, 'v': 56, 'w': 57, 'x': 58, 'y': 59, 'z': 60}
['\n' ' ' '!' "'" '(' ')' ',' '-' '.' ':' ';' '?' 'A' 'B' 'C' 'D' 'E' 'F'
 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'R' 'S' 'T' 'U' 'V' 'W' 'Y' 'a'
 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o' 'p' 'q' 'r' 's'
 't' 'u' 'v' 'w' 'x' 'y' 'z']


In [17]:
int_text.shape

(94238,)

## Create Training Data

In [18]:
seq_length= 40 

examples_per_epoch = len(text)

char_dataset = tf.data.Dataset.from_tensor_slices(int_text)

In [19]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [22]:
# sequences

<BatchDataset shapes: (41,), types: tf.int64>

In [23]:
#Testing
print("Character Stream: \n")
for i in char_dataset.take(10):
  print(index_to_char[i.numpy()])  

print("\nSequence: \n")
for i in sequences.take(10):
  print(repr(''.join(index_to_char[i.numpy()])))  #use repr() for more clarity. str() keeps formatting it

Character Stream: 

F
r
o
m
 
f
a
i
r
e

Sequence: 

'From fairest creatures we desire increase'
",\nThat thereby beauty's rose might never "
'die,\nBut as the riper should by time dece'
'ase,\nHis tender heir might bear his memor'
'y:\nBut thou contracted to thine own brigh'
"t eyes,\nFeed'st thy light's flame with se"
'lf-substantial fuel,\nMaking a famine wher'
'e abundance lies,\nThy self thy foe, to th'
'y sweet self too cruel:\nThou that art now'
" the world's fresh ornament,\nAnd only her"



Target value: for each sequence of characters, we return that sequence, shifted one position to the right, along with the new character that is predicted to follow the sequence.

To create training examples of (input, target) pairs, we take the given sequence. The input is sequence with last word removed. Target is sequence with first word removed. Example: sequence: abc d ef input: abc d e target: bc d ef

In [25]:
def create_input_target_pair(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(create_input_target_pair)

In [26]:
#Testing
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(index_to_char[input_example.numpy()])))
  print ('Target data:', repr(''.join(index_to_char[target_example.numpy()])))

Input data:  'From fairest creatures we desire increas'
Target data: 'rom fairest creatures we desire increase'


In [27]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 40), (64, 40)), types: (tf.int64, tf.int64)>

\## Building the Model

In [28]:
vocab_size = len(vocabulary)
embedding_dim = 256
rnn_units= 1024

3 Layers used:

Input Layer: Maps character to 256 dimension vector

GRU Layer: RNN of size 1024

Dense Layer: Output with same size as vocabulary

Since it is a character level RNN, we can use keras.Sequential model (All layers have single input and single output).

In [29]:
def build_model_lstm(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units, 
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
    return model

# Reference for theory: https://jhui.github.io/2017/03/15/RNN-LSTM-GRU/

In [30]:
lstm_model = build_model_lstm(
  vocab_size = vocab_size,
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [31]:
#Testing: shape
for input_example_batch, target_example_batch in dataset.take(1):
    example_prediction = lstm_model(input_example_batch)
    assert (example_prediction.shape == (BATCH_SIZE, seq_length, vocab_size)), "Shape error"
    #print(example_prediction.shape)

In [32]:
#model.summary() 
#check shapes if necessary

In [33]:
sampled_indices = tf.random.categorical(example_prediction[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

## Model Training

In [34]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

#Loss Function reference: https://www.dlology.com/blog/how-to-use-keras-sparse_categorical_crossentropy/

example_loss  = loss(target_example_batch, example_prediction)
print("Prediction shape: ", example_prediction.shape)
print("Loss:      ", example_loss.numpy().mean())

Prediction shape:  (64, 40, 61)
Loss:       4.109836


In [35]:
lstm_model.compile(optimizer='adam', loss=loss)

In [36]:
lstm_dir_checkpoints= './training_checkpoints_LSTM'
checkpoint_prefix = os.path.join(lstm_dir_checkpoints, "checkpt_{epoch}") #name
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,save_weights_only=True)

In [38]:
EPOCHS=100 #increase number of epochs for better results (lesser loss)

In [39]:
history = lstm_model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
35/35 [==============================] - 1s 28ms/step - loss: 3.3371
Epoch 2/100
35/35 [==============================] - 1s 26ms/step - loss: 2.8657
Epoch 3/100
35/35 [==============================] - 1s 26ms/step - loss: 2.5391
Epoch 4/100
35/35 [==============================] - 1s 26ms/step - loss: 2.3212
Epoch 5/100
35/35 [==============================] - 1s 26ms/step - loss: 2.2272
Epoch 6/100
35/35 [==============================] - 1s 26ms/step - loss: 2.1615
Epoch 7/100
35/35 [==============================] - 1s 25ms/step - loss: 2.0945
Epoch 8/100
35/35 [==============================] - 1s 25ms/step - loss: 2.0323
Epoch 9/100
35/35 [==============================] - 1s 27ms/step - loss: 1.9719
Epoch 10/100
35/35 [==============================] - 1s 26ms/step - loss: 1.9201
Epoch 11/100
35/35 [==============================] - 1s 28ms/step - loss: 1.8725
Epoch 12/100
35/35 [==============================] - 1s 26ms/step - loss: 1.8254
Epoch 13/100
35/35 [=====

In [40]:
tf.train.latest_checkpoint(lstm_dir_checkpoints)

'./training_checkpoints_LSTM/checkpt_100'

## Prediction

In [41]:
lstm_model = build_model_lstm(vocab_size, embedding_dim, rnn_units, batch_size=1)
lstm_model.load_weights(tf.train.latest_checkpoint(lstm_dir_checkpoints))
lstm_model.build(tf.TensorShape([1, None]))

lstm_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            15616     
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_1 (Dense)              (1, None, 61)             62525     
Total params: 5,325,117
Trainable params: 5,325,117
Non-trainable params: 0
_________________________________________________________________


In [42]:
def generate_text(model, start_string):
    num_generate = 1000 #Number of characters to be generated

    input_eval = [char_to_index[s] for s in start_string] #vectorising input
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.5

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(index_to_char[predicted_id])

    return (start_string + ''.join(text_generated))

In [43]:
#Testing
#print(generate_text(lstm_model, start_string=u"ROMEO: "))

In [44]:
#Prediction with User Input
lstm_test = input("Enter your starting string: ")
print(generate_text(lstm_model, start_string=lstm_test))


Enter your starting string:  You live


You live in this poor rhyme,
While he will not every hour survey,
For blunting age dead?
No, neither he, nor his compeers bareness every where:
Thou mayst call thine own self bring:
And what is's deceives repose laid to make the taker mad.
Mad in perfection but a little moment.
That thou in losing me, shalt win much outly gays,
And make me bow,
And do not drop in for my sin, grounded on sinful loving,
O if (I say) you look upon this verse,
As every alien pen hath got my use it might unused stay
From far worsing things:
Alas why fearing of time's ty my sun one early morn did shine eye,
When love converted from thee,  
Whilst I thou wilt, for I being pent in thee,
Per for my love, not for their rhyme,
Exce compound sweet; forgoing simple savour,
Which huld all my poor beast then find,
When swift eyes can see,
So long lives this, and disgrace.
Therefore my mistress' eyes are restored in the basest clouds to ride,
With ught,
In dis a healthful remedy,
For men discased, but shall carry me a